<a href="https://colab.research.google.com/github/IamMarking/Mark/blob/master/3_2_MNIST_%E7%AE%80%E5%8D%95%E5%88%86%E7%B1%BB%E7%89%88%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[神经网络学习](https://www.zybuluo.com/hanbingtao/note/433855)

In [7]:
import tensorflow as tf


from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("sample_data/", one_hot=True)

Extracting sample_data/train-images-idx3-ubyte.gz
Extracting sample_data/train-labels-idx1-ubyte.gz
Extracting sample_data/t10k-images-idx3-ubyte.gz
Extracting sample_data/t10k-labels-idx1-ubyte.gz


In [0]:
tf.Session()

调试代码：
- batch_size = 100 可以修改
- 添加隐藏层
- 激活函数
- 初始化的值
- 损失函数交叉熵
- 优化器学习率的调整
- 训练次数

In [22]:
import tensorflow as tf

batch_size = 100  # 1 次放入 100 张图片进行训练，每张图片维度是 784 维

n_batch = mnist.train.num_examples // batch_size  # 计算一共有多少个批次

x = tf.placeholder(tf.float32,[None,784])   
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# 构建一个简单的神经网络
# W = tf.Variable(tf.zeros([784,10]))
W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1))
b = tf.Variable(tf.zeros([10]))

prediction = tf.nn.softmax(tf.matmul(x,W)+b)  # 100 * 784 * 784 * 10 = 100 * 10 * 10 * 1 = 100 * 1

loss = tf.reduce_mean(tf.square(y-prediction))  # mse 
# loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)  # 交叉熵损失

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))  # argmax 返回一维张量中最大的值所在的位置

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  # 转换类型，由布尔型到 float32

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):  # 训练 21 次
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)  # 每 100 张依次进行训练
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})  # 每一次的准确率变化
        print('Iter ' + str(epoch) + ', Testing Accuracy ' + str(acc) )


Iter 0, Testing Accuracy 0.5795
Iter 1, Testing Accuracy 0.7265
Iter 2, Testing Accuracy 0.806
Iter 3, Testing Accuracy 0.8425
Iter 4, Testing Accuracy 0.8588
Iter 5, Testing Accuracy 0.8683
Iter 6, Testing Accuracy 0.8755
Iter 7, Testing Accuracy 0.8794
Iter 8, Testing Accuracy 0.8849
Iter 9, Testing Accuracy 0.8868
Iter 10, Testing Accuracy 0.8889
Iter 11, Testing Accuracy 0.8906
Iter 12, Testing Accuracy 0.892
Iter 13, Testing Accuracy 0.8934
Iter 14, Testing Accuracy 0.8952
Iter 15, Testing Accuracy 0.8952
Iter 16, Testing Accuracy 0.8979
Iter 17, Testing Accuracy 0.8988
Iter 18, Testing Accuracy 0.8999
Iter 19, Testing Accuracy 0.9007
Iter 20, Testing Accuracy 0.9013


In [53]:
batch_size = 128  # 1 次放入 100 张图片进行训练，每张图片维度是 784 维

n_batch = mnist.train.num_examples // batch_size  # 计算一共有多少个批次

x = tf.placeholder(tf.float32,[None,784])   
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# 构建一个简单的神经网络
# W = tf.Variable(tf.zeros([784,10]))
W1 = tf.Variable(tf.truncated_normal([784,200],stddev=0.1))  # 初始化权重
b1 = tf.Variable(tf.zeros([200])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

W2 = tf.Variable(tf.truncated_normal([200,200],stddev=0.1))
b2 = tf.Variable(tf.zeros([200])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.truncated_normal([200,100],stddev=0.1))
b3 = tf.Variable(tf.zeros([100])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop = tf.nn.dropout(L3,keep_prob)


W4 = tf.Variable(tf.truncated_normal([100,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)  # 100 * 784 * 784 * 10 = 100 * 10 * 10 * 1 = 100 * 1

# loss = tf.reduce_mean(tf.square(y-prediction))  # mse 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))  # 交叉熵损失

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))  # argmax 返回一维张量中最大的值所在的位置

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  # 转换类型，由布尔型到 float32

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):  # 训练 21 次
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)  # 每 100 张依次进行训练
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})  # 每一次的准确率变化
        print('Iter ' + str(epoch) + ', Testing Accuracy ' + str(test_acc) + ', Training Accuracy ' + str(train_acc))

Iter 0, Testing Accuracy 0.9271, Training Accuracy 0.92925453
Iter 1, Testing Accuracy 0.9458, Training Accuracy 0.94963634
Iter 2, Testing Accuracy 0.9544, Training Accuracy 0.9604
Iter 3, Testing Accuracy 0.959, Training Accuracy 0.9671091
Iter 4, Testing Accuracy 0.9639, Training Accuracy 0.9724727
Iter 5, Testing Accuracy 0.9642, Training Accuracy 0.97547275
Iter 6, Testing Accuracy 0.9682, Training Accuracy 0.97714543
Iter 7, Testing Accuracy 0.9686, Training Accuracy 0.98030907
Iter 8, Testing Accuracy 0.9689, Training Accuracy 0.98212725
Iter 9, Testing Accuracy 0.9716, Training Accuracy 0.984
Iter 10, Testing Accuracy 0.9724, Training Accuracy 0.9852727
Iter 11, Testing Accuracy 0.9726, Training Accuracy 0.98563635
Iter 12, Testing Accuracy 0.9737, Training Accuracy 0.98718184
Iter 13, Testing Accuracy 0.9729, Training Accuracy 0.98678184
Iter 14, Testing Accuracy 0.9745, Training Accuracy 0.9888909
Iter 15, Testing Accuracy 0.9759, Training Accuracy 0.9892909
Iter 16, Testing 